# Anomaly Detection (Binary 3D)

Discuss dataset here including shape. Also discuss which data set you specifically chose here.


**Inputs**  


The 14 waveforms (features) reported in the dataset in order are:

- `A+IGBT-I`: Current passing through the IGBT switch of phase A+ in Qa1 ($A$).  
- `A+*IGBT-I`: Current passing through the IGBT switch of phase A+\* in Qa3 ($A$).  
- `B+IGBT-I`: Current passing through the IGBT switch of phase B+ in Qb1 ($A$).  
- `B+*IGBT-I`: Current passing through the IGBT switch of phase B+\* in Qb3 ($A$).  
- `C+IGBT-I`: Current passing through the IGBT switch of phase C+ in Qc1 ($A$).  
- `C+*IGBT-I`: Current passing through the IGBT switch of phase C+\* in Qc3 ($A$).  
- `A-Flux`: Magnetic flux density for phase A in transformer XA ($-$).  
- `B-Flux`: Magnetic flux density for phase B in transformer XB ($-$).  
- `C-Flux`: Magnetic flux density for phase C in transformer XC ($-$).  
- `Mod-V`: Modulator voltage ($V$).  
- `Mod-I`: Modulator current ($A$).  
- `CB-I`: Cap bank current ($-$).  
- `CB-V`: Cap bank voltage ($V$).  
- `DV/DT`: Time derivative change of the Mod\-V voltage ($-$).

**Outputs**
- `Fault`: Faulty wave pulse
- `Normal` Normal wave pulse

![Image](display.jpg)


e (unit: -).


**Files and Descriptions**

The files provided in the dataset are:

- `RFQ.npy`: 3D numpy array of the normal and faulty pulses for the RFQ system (shape: (872, 4500, 14)).
- `RFQ_labels.npy`: 2D numpy array of the labels for the RFQ system (shape: (872, 3)).
- `DTL.npy`: 3D numpy array of the normal and faulty pulses for the DTL system (shape: (1077, 4500, 14)).
- `DTL_labels.npy`: 2D numpy array of the labels for the DTL system (shape: (1077, 3)).
- `CCL.npy`: 3D numpy array of the normal and faulty pulses for the CCL system (shape: (2057, 4500, 14)).
- `CCL_labels.npy`: 2D numpy array of the labels for the CCL system (shape: (2057, 3)).
- `SCL.npy`: 3D numpy array of the normal and faulty pulses for the SCL system (shape: (4598, 4500, 14)).
- `SCL_labels.npy`: 2D numpy array of the labels for the SCL system (shape: (4598, 3)).

High Voltage Converter Modulators (HVCM) have frequent failures, making them a major source of down time for the Spallation neutron source. HVCMs are well instrumented to collect large amounts of waveform data including but not limited to modulator current, modulator voltage, magnetic flux, cap bank voltage, cap bank current, and others. These waveform data are collected from all 15 HVCM systems for the application of anomaly detection and failure prediction.

All 15 modulators are grouped into four major subsystems based on their type as follows: 1 modulator as radio-frequency quadrupole (RFQ), 2 modulators as drift-tube linac (DTL), 4 modulators as coupled-cavity linac (CCL), and 8 modulators as super-conducting linac (SCL). A waveform is featured by consecutive pulses, which could be either a normal pulse if the system is healthy, or anomaly pulse if the system is close to failing. These pulses are collected and reported in the dataset of this paper, where the details of the data collection and processing are described in the next section.

The researchers will select the DTL dataset for this example.

4598, 3)

In [21]:
%load_ext autoreload
%autoreload 2

import pickle
import sys
sys.path.insert(0, "supporting/anomaly")

from supporting.anomaly.preprocessing import load_anomaly_data
import supporting.anomaly.settings as anomaly_settings
import pyMAISE as mai

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## pyMAISE Initialization

Discuss initialization: settings file, use of GPU for LSTM/GRU.

In [12]:
_ = mai.init(
    problem_type=anomaly_settings.problem_type,
    verbosity=anomaly_settings.verbosity,
    random_state=anomaly_settings.random_state,
    cuda_visible_devices="1",  # Use GPU 1
)

Num GPUs Available:  1


Discuss train test split, input data scaling, and the external file we used. Also removal of some positive cases and discuss frequency plot.

![Frequency of positive/negative values in anomaly detection data.](supporting/anomaly/figs/bc1_frequency.png)

In [13]:
xtrain, xtest, ytrain, ytest, xscaler = load_anomaly_data(
    stack_series=False,
    multiclass=False,
    test_size=anomaly_settings.test_size,
    non_faulty_frac=anomaly_settings.non_faulty_frac,
    timestep_step=1,
)

xtrain shape: (239, 4500, 14)
xtest shape: (103, 4500, 14)
ytrain shape: (239, 2)
ytest shape: (103, 2)


In [14]:
xtrain

<xarray.DataArray (samples: 239, time steps: 4500, features: 14)>
array([[[0.60112906, 0.50060827, 0.8019305 , ..., 0.500683  ,
         0.9550177 , 0.49246472],
        [0.60112906, 0.4990089 , 0.80275345, ..., 0.500683  ,
         0.95524377, 0.49002475],
        [0.60112906, 0.49932876, 0.802342  , ..., 0.500683  ,
         0.9551081 , 0.49368486],
        ...,
        [0.6057133 , 0.49804923, 0.80522215, ..., 0.49976605,
         0.920795  , 0.5071064 ],
        [0.6054076 , 0.49772674, 0.80563366, ..., 0.500683  ,
         0.9190771 , 0.5010045 ],
        [0.60449076, 0.49804923, 0.80522215, ..., 0.498849  ,
         0.91921276, 0.5010045 ]],

       [[0.5882934 , 0.69393533, 0.79987323, ..., 0.69784105,
         0.9644662 , 0.4631784 ],
        [0.5882934 , 0.6654474 , 0.80069613, ..., 0.71158785,
         0.9643307 , 0.4509795 ],
        [0.58707094, 0.63920134, 0.79863894, ..., 0.7244332 ,
         0.96487314, 0.45708147],
...
        [0.603327  , 0.6107938 , 0.8012654 , ..., 0.49976605,
         0.9191223 , 0.50147426],
        [0.6030213 , 0.611434  , 0.8024998 , ..., 0.498849  ,
         0.91867024, 0.50269365],
        [0.603327  , 0.61111397, 0.80044246, ..., 0.498849  ,
         0.91844416, 0.50269365]],

       [[0.5913494 , 0.71538055, 0.79863894, ..., 0.66895026,
         0.9549273 , 0.43635184],
        [0.5910438 , 0.68817484, 0.7982274 , ..., 0.67766404,
         0.95411354, 0.4375612 ],
        [0.58951586, 0.6622486 , 0.7974043 , ..., 0.68959296,
         0.9538875 , 0.44243878],
        ...,
        [0.60265714, 0.6116752 , 0.80357635, ..., 0.49976605,
         0.918625  , 0.48148426],
        [0.60112906, 0.6116752 , 0.80151904, ..., 0.49976605,
         0.9179469 , 0.48392802],
        [0.60143465, 0.6116752 , 0.80151904, ..., 0.498849  ,
         0.91853464, 0.48514244]]], dtype=float32)
Coordinates:
  * samples     (samples) int64 183 122 21 167 17 136 ... 43 137 36 15 115 144
  * time steps  (time steps) int64 0 1 2 3 4 5 ... 4494 4495 4496 4497 4498 4499
  * features    (features) <U18 'A+IGBT-I: current' ... 'DV/DT'

In [15]:
ytrain

<xarray.DataArray (samples: 239, features: 2)>
array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
...
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])
Coordinates:
  * samples   (samples) int64 183 122 21 167 17 136 191 ... 43 137 36 15 115 144
  * features  (features) object 'Class_Fault' 'Class_Run'

## Model Initialization and Hyperparameter Tuning

Discuss the models we accessed, their hyperparameter search spaces, and dumping/loading of pickled configurations. Discuss convergence plot below too.

![Convergence of Bayesian optimizer to best hyperparameter configuration.](supporting/anomaly/figs/bc1_convergence.png)

In [16]:
with open("supporting/anomaly/configs/binary_case_1.pkl", "rb") as f:
    configs = pickle.load(f)

## Model Postprocessing

Discuss post processing, including an increase of epochs for both models from 5 to 100.

In [17]:
postprocessor = mai.PostProcessor(
    data=(xtrain, xtest, ytrain, ytest),
    model_configs=[configs],
    new_model_settings={
        "LSTM": {"fitting_params": {"epochs": 100}},
        "GRU": {"fitting_params": {"epochs": 100}},
    },
)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

Discuss performance metrics. Include equations and definitions of each metric.

In [18]:
postprocessor.metrics()

,Model Types,Parameter Configurations,Train Accuracy,Train Recall,Train Precision,Train F1,Test Accuracy,Test Recall,Test Precision,Test F1
1,LSTM,"{'LSTM_input_0_units': 148, 'LSTM_num_layers':...",0.736402,0.698690,0.789581,0.697195,0.611650,0.636537,0.666822,0.600775
2,LSTM,"{'LSTM_input_0_units': 113, 'LSTM_num_layers':...",0.665272,0.607843,0.815668,0.564425,0.572816,0.614035,0.755556,0.523950
4,LSTM,"{'LSTM_input_0_units': 148, 'LSTM_num_layers':...",0.673640,0.617647,0.818605,0.579681,0.572816,0.614035,0.755556,0.523950
8,GRU,"{'LSTM_input_0_units': 148, 'LSTM_num_layers':...",0.665272,0.607843,0.815668,0.564425,0.572816,0.614035,0.755556,0.523950
3,LSTM,"{'LSTM_input_0_units': 55, 'LSTM_num_layers': ...",0.732218,0.707564,0.741226,0.710829,0.563107,0.575896,0.578968,0.561619
5,GRU,"{'LSTM_input_0_units': 55, 'LSTM_num_layers': ...",0.698745,0.649563,0.799857,0.630031,0.563107,0.598970,0.669608,0.528244
7,GRU,"{'LSTM_input_0_units': 113, 'LSTM_num_layers':...",0.656904,0.598039,0.812785,0.548766,0.563107,0.605263,0.752747,0.509679
9,GRU,"{'LSTM_input_0_units': 101, 'LSTM_num_layers':...",0.648536,0.588235,0.809955,0.532682,0.553398,0.596491,0.750000,0.495098
0,LSTM,"{'LSTM_input_0_units': 101, 'LSTM_num_layers':...",0.694561,0.688493,0.688035,0.688252,0.524272,0.521930,0.521731,0.521385
6,GRU,"{'LSTM_input_0_units': 112, 'LSTM_num_layers':...",0.640167,0.654823,0.655223,0.640161,0.524272,0.503051,0.503584,0.491896


Discuss the results and information provided in the performance metrics.

In [19]:
for model in ["LSTM", "GRU"]:
    for key, value in postprocessor.get_params(model_type=model).to_dict().items():
        print(f"{key}: {value[0]}")
    print()

Model Types: LSTM
LSTM_input_0_units: 148
LSTM_num_layers: 0
LSTM_output_0_units: 94
LSTM_output_0_activation: tanh
Dense_num_layers: 0
Adam_learning_rate: 0.0005765861336545322
Adam_clipnorm: 0.9562849717510996
Adam_clipvalue: 0.6580160563721252
Dense_0_units: 80
Dense_1_units: 78
Dense_2_units: 155
Dense_3_units: 225
batch_size: 16
LSTM_0_units: 93
LSTM_0_activation: sigmoid
LSTM_1_units: 44
LSTM_1_activation: sigmoid
LSTM_2_units: 65
LSTM_2_activation: tanh
LSTM_3_units: 115
LSTM_3_activation: tanh

Model Types: GRU
LSTM_input_0_units: 148
LSTM_num_layers: 0
LSTM_output_0_units: 70
LSTM_output_0_activation: tanh
Dense_num_layers: 0
Adam_learning_rate: 0.00031861581260432347
Adam_clipnorm: 1.1848527213970947
Adam_clipvalue: 0.665867812466699
Dense_0_units: 52
Dense_1_units: 72
Dense_2_units: 93
Dense_3_units: 232
batch_size: 8
LSTM_0_units: 129
LSTM_0_activation: tanh
LSTM_1_units: 32
LSTM_1_activation: sigmoid
LSTM_2_units: 147
LSTM_2_activation: tanh
LSTM_3_units: 75
LSTM_3_activat

Discuss anything interesting (if anything) about the hyperparameter configurations.

In [20]:
postprocessor.confusion_matrix(model_type="LSTM")

ValueError: multilabel-indicator is not supported

In [ ]:
postprocessor.confusion_matrix(model_type="GRU")

Discuss confusion matrix results.

In [ ]:
postprocessor.nn_learning_plot(model_type="LSTM")

In [ ]:
postprocessor.nn_learning_plot(model_type="GRU")

Discuss neural network learning curves.

![pyMAISE Logo](../docs/source/_images/pyMAISElogo.png)